In [77]:
!pip install meteostat
!pip install geocoder
!pip install geopy

In [78]:
import pandas as pd
import json
import ast
import geocoder
from geopy.geocoders import Nominatim
from datetime import datetime
import matplotlib.pyplot as plt
from meteostat import Point, Hourly, Daily, Monthly

In [79]:
# Acceso a gdrive para datos
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/ProyectoAI/datasets/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [80]:
dfconsumption = pd.read_csv(path + "/consumptions_small.csv", sep=';')
dfcups = pd.read_csv(path + "/cups_info_small.csv", sep=';')

In [81]:
# Normalizamos los datos de consumo
dfconsumption.consumption = dfconsumption.consumption.apply(lambda x: ast.literal_eval(x.replace("{", "(").replace("}", ")")))
dfconsumption['total_consumption'] = dfconsumption.consumption.apply(lambda x: sum(x))

In [82]:
dfconsumption.head()

,day,consumption,consumption_type,id,total_consumption
0,2019-12-01,"(1.139, 2.04)","{TELEM,TELEM}",39653,3.179
1,2019-12-02,"(1.163, 2.328)","{TELEM,TELEM}",39653,3.491
2,2019-12-03,"(0.873, 2.578)","{TELEM,TELEM}",39653,3.451
3,2019-12-04,"(0.997, 3.089)","{TELEM,TELEM}",39653,4.086
4,2019-12-05,"(1.818, 3.694)","{TELEM,TELEM}",39653,5.512


In [83]:
dfcups.head()

,id,atr,power,zipcode,locality,ekon_start
0,39653,2.0TD,"{4600,4600}",28052,NaN,2021-09-01


In [84]:
# Hacemos join con los datos de las dos tablas
dfmerged = pd.merge(left=dfconsumption, right=dfcups, left_on='id', right_on='id')
dfmerged.head()

,day,consumption,consumption_type,id,total_consumption,atr,power,zipcode,locality,ekon_start
0,2019-12-01,"(1.139, 2.04)","{TELEM,TELEM}",39653,3.179,2.0TD,"{4600,4600}",28052,NaN,2021-09-01
1,2019-12-02,"(1.163, 2.328)","{TELEM,TELEM}",39653,3.491,2.0TD,"{4600,4600}",28052,NaN,2021-09-01
2,2019-12-03,"(0.873, 2.578)","{TELEM,TELEM}",39653,3.451,2.0TD,"{4600,4600}",28052,NaN,2021-09-01
3,2019-12-04,"(0.997, 3.089)","{TELEM,TELEM}",39653,4.086,2.0TD,"{4600,4600}",28052,NaN,2021-09-01
4,2019-12-05,"(1.818, 3.694)","{TELEM,TELEM}",39653,5.512,2.0TD,"{4600,4600}",28052,NaN,2021-09-01


In [85]:
# Necesitamos encontrar todos los diferentes zipcode del dataset y con sus fechas mínimas y máximas para la obtención de datos meteorológicos
df_zipcodes = dfmerged.groupby('zipcode').agg({'day': ['min', 'max']})
display(df_zipcodes.head())
df_zipcodes.info()

day            
                min         max
zipcode                        
28052    2019-12-01  2021-05-31

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 28052 to 28052
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   (day, min)  1 non-null      object
 1   (day, max)  1 non-null      object
dtypes: object(2)
memory usage: 24.0+ bytes


In [86]:
# Recorremos todos los zipcode y poblamos un nuevo dataset con zipcode, fechas y datos meteorológicos diarios
geolocator = Nominatim(user_agent='test')
df = pd.DataFrame(columns=['time', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun', 'zipcode'])
for index, x in df_zipcodes.iterrows():
  location = geolocator.geocode(str(x.name) + ", Spain")
  point = Point(location.latitude, location.longitude)
  data = Daily(point, pd.to_datetime(x.day.min()), pd.to_datetime(x.day.max()))
  data = data.fetch()
  # Obtenemos los datos de luz diaria con los datos mensuales
  dataMonth = Monthly(point, pd.to_datetime(x.day.min()), pd.to_datetime(x.day.max()))
  dataMonth = dataMonth.fetch()
  dataMonth = dataMonth.reset_index()
  # data.plot(y=['tavg', 'tmin', 'tmax'])
  data['zipcode'] = x.name
  data = data.reset_index()
  data['month'] = pd.to_datetime(data.time).dt.month
  data['year'] = pd.to_datetime(data.time).dt.year
  dataMonth['month'] = pd.to_datetime(dataMonth.time).dt.month
  dataMonth['year'] = pd.to_datetime(dataMonth.time).dt.year
  data = pd.merge(
    data, 
    dataMonth[['tsun', 'month', 'year']], 
    on=['month', 'year'],
    how='left')
  data['tsun'] = data['tsun_y']/30
  data = data.drop(['month', 'year', 'tsun_x', 'tsun_y'], axis = 1)
  df = df.append(data)
  print(pd.to_datetime(x.day.min()))
  print(pd.to_datetime(x.day.max()))
  print(location.address)
df_zipcodes.head()

# Los datos ya estarían listos para hacer join por el zipcode con los datos del consumer



2019-12-01 00:00:00
2021-05-31 00:00:00
Puente de Vallecas, Madrid, Área metropolitana de Madrid y Corredor del Henares, Comunidad de Madrid, 28052, España


day            
                min         max
zipcode                        
28052    2019-12-01  2021-05-31

In [87]:
# Tenemos que transformar el valor object a datetime
dfmerged['day'] =  pd.to_datetime(dfmerged['day'])
# Unimos las dos tablas a través del zipcode y la fecha
dfmerged = pd.merge(left=dfmerged, right=df, left_on=['zipcode', 'day'], right_on=['zipcode', 'time'], how='left')
dfmerged.head()

,day,consumption,consumption_type,id,total_consumption,atr,power,zipcode,locality,ekon_start,time,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
0,2019-12-01,"(1.139, 2.04)","{TELEM,TELEM}",39653,3.179,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-01,7.4,6.2,8.7,26.8,NaN,71.2,7.8,NaN,1013.1,276.0
1,2019-12-02,"(1.163, 2.328)","{TELEM,TELEM}",39653,3.491,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-02,7.4,5.3,10.8,0.0,NaN,25.3,15.5,NaN,1016.1,276.0
2,2019-12-03,"(0.873, 2.578)","{TELEM,TELEM}",39653,3.451,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-03,5.3,2.8,9.0,2.0,NaN,11.2,10.7,NaN,1015.6,276.0
3,2019-12-04,"(0.997, 3.089)","{TELEM,TELEM}",39653,4.086,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-04,7.8,5.4,11.6,2.0,NaN,6.4,10.6,NaN,1011.5,276.0
4,2019-12-05,"(1.818, 3.694)","{TELEM,TELEM}",39653,5.512,2.0TD,"{4600,4600}",28052,NaN,2021-09-01,2019-12-05,9.3,7.9,12.0,0.0,NaN,10.9,12.0,NaN,1017.6,276.0
